# Parsing

Points to clarify:
* Is empty 'contracts' field a valid case and what does it mean if it is null? For now, we will assume it is a valid case and we handle it as -3.
* Is it possible that 'day_sinlastloan' is negative number? For now, we assume that it is possible and might mean that the loan period ends in the future.
* If date is in an invalid format we assume that we should not take into consideration such dates and we will handle it as -3.
* If the 'claim_date' field is null we assume that we should not take into consideration such claims.
* If the 'loan_summa' is null we assume that we should not take into consideration such loans.
* If the 'contract_date' is null we assume that we should not take into consideration such loans.


Notes:
* error_log.csv file should be substituted with logging module in production code.

In [2]:
import pandas as pd
import json
import math
from datetime import timezone
from dateutil import parser

In [3]:
df = pd.read_csv('data.csv')
df.head()

,id,application_date,contracts
0,2925210.0,2024-02-12 19:22:46.652000+00:00,NaN
1,2925211.0,2024-02-12 19:24:29.135000+00:00,"[{""contract_id"": 522530, ""bank"": ""003"", ""summa..."
2,2925212.0,2024-02-12 19:24:41.493000+00:00,NaN
3,2925213.0,2024-02-12 19:24:29.135000+00:00,"[{""contract_id"": 522530, ""bank"": ""003"", ""summa..."
4,2925214.0,2024-02-12 19:24:56.857000+00:00,NaN


In [4]:
df.shape

(1000, 3)

In [5]:
df.isna().sum()

id                    0
application_date      0
contracts           505
dtype: int64

In [6]:
df['contracts'] = df['contracts'].apply(lambda x: json.loads(x) if not pd.isna(x) else x)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                1000 non-null   float64
 1   application_date  1000 non-null   object 
 2   contracts         495 non-null    object 
dtypes: float64(1), object(2)
memory usage: 23.6+ KB


In [7]:
ERROR_LOG = pd.DataFrame(columns=['value', 'error'])

def ensure_datetime_aware(dt_str):
    """Parses a datetime string and ensures it's timezone-aware (UTC).

    Automatically detects the format and converts it to UTC.
    If the datetime is naive, it assumes it's in UTC.
    """
    dt = parser.parse(dt_str)
    if dt.tzinfo is None:
        dt = dt.replace(tzinfo=timezone.utc)
    return dt.astimezone(timezone.utc)

def calc_tot_claim_cnt_l180d(contracts, application_date):
    """Calculates the number of claims for the last 180 days."""
    if not contracts or (isinstance(contracts, float) and math.isnan(contracts)):
        return -3

    try:
        application_date = ensure_datetime_aware(application_date)
    except Exception as invalid_application_date_error:
        ERROR_LOG.append({'value': application_date, 'error': invalid_application_date_error})

        return -3

    claim_count = 0

    for contract in contracts:
        if not contract or not isinstance(contract, dict):
            continue
        
        claim_date = contract.get('claim_date')
        if claim_date and isinstance(claim_date, str):
            try:
                claim_date = ensure_datetime_aware(claim_date)
                if (application_date - claim_date).days <= 180:
                    claim_count += 1
            except Exception as invalid_claim_date_error:
                ERROR_LOG.append({'value': application_date, 'error': invalid_claim_date_error})

                continue
    
    return claim_count if claim_count > 0 else -3

def calc_disb_bank_loan_wo_tbc(contracts):
    """Calculates the sum of loans without TBC loans."""
    if not contracts or (isinstance(contracts, float) and math.isnan(contracts)):
        return -1
    
    excluded_banks = ['LIZ', 'LOM', 'MKO', 'SUG']
    total_exposure = 0
    has_loans = False

    for contract in contracts:
        if not contract or not isinstance(contract, dict):
            continue
        
        bank = contract.get('bank')
        loan_summa = contract.get('loan_summa')
        contract_date = contract.get('contract_date')
        
        if bank in excluded_banks or bank is None or (isinstance(bank, float) and math.isnan(bank)):
            continue
        
        if contract_date is None or (isinstance(contract_date, float) and math.isnan(contract_date)):
            continue
        if not loan_summa or (isinstance(loan_summa, str) and not loan_summa.strip()):
            continue

        try:
            total_exposure += float(loan_summa)
            has_loans = True
        except (ValueError, TypeError) as invalid_loan_summa_error:
            ERROR_LOG.append({'value': loan_summa, 'error': invalid_loan_summa_error})
            
            continue

    if not has_loans:
        return -1
    return total_exposure if total_exposure > 0 else -3

def calc_days_since_lastloan(contracts, application_date):
    """Calculates the number of days since the last loan."""
    if not contracts or (isinstance(contracts, float) and math.isnan(contracts)):
        return -1

    try:
        application_date = ensure_datetime_aware(application_date)
    except Exception as invalid_application_date_error:
        ERROR_LOG.append({'value': application_date, 'error': invalid_application_date_error})
        
        return -3

    last_loan_date = None

    for contract in contracts:
        if not contract or not isinstance(contract, dict):
            continue

        contract_date = contract.get('contract_date')
        summa = contract.get('summa')
        
        if not contract_date or (isinstance(contract_date, str) and not contract_date.strip()):
            continue
        if not summa or (isinstance(summa, str) and not summa.strip()):
            continue

        try:
            contract_date = ensure_datetime_aware(contract_date)
            if last_loan_date is None or contract_date > last_loan_date:
                last_loan_date = contract_date
        except Exception as invalid_contract_date_error:
            ERROR_LOG.append({'value': contract_date, 'error': invalid_contract_date_error})
            
            continue

    if last_loan_date:
        return (application_date - last_loan_date).days
    return -3 if any(contract.get('summa') for contract in contracts if isinstance(contract, dict)) else -1

In [8]:
df['tot_claim_cnt_l180d'] = df.apply(lambda row: calc_tot_claim_cnt_l180d(row['contracts'], row['application_date']), axis=1)
df['disb_bank_loan_wo_tbc'] = df.apply(lambda row: calc_disb_bank_loan_wo_tbc(row['contracts']), axis=1)
df['day_sinlastloan'] = df.apply(lambda row: calc_days_since_lastloan(row['contracts'], row['application_date']), axis=1)

ERROR_LOG.to_csv('error_log.csv', index=False)

In [9]:
contract_features = df[['id','application_date', 'tot_claim_cnt_l180d', 'disb_bank_loan_wo_tbc', 'day_sinlastloan']]
contract_features.shape

(1000, 5)

In [10]:
contract_features.to_csv('contract_features.csv', index=False)
contract_features.head()

,id,application_date,tot_claim_cnt_l180d,disb_bank_loan_wo_tbc,day_sinlastloan
0,2925210.0,2024-02-12 19:22:46.652000+00:00,-3,-1.0,-1
1,2925211.0,2024-02-12 19:24:29.135000+00:00,60,-1.0,427
2,2925212.0,2024-02-12 19:24:41.493000+00:00,-3,-1.0,-1
3,2925213.0,2024-02-12 19:24:29.135000+00:00,60,-1.0,427
4,2925214.0,2024-02-12 19:24:56.857000+00:00,-3,-1.0,-1
